In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
import tensorflow as tf

In [3]:
import numpy as np

In [4]:
import random

In [5]:
config=tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
tf.set_random_seed(777)  # reproducibility

In [7]:
# parameters
log_dir = './logs/ML_Lab_10-01-Softmax'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [8]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
print(mnist.train.num_examples)

55000


In [10]:
nb_classes = 10

In [11]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 784] , name='X')

In [12]:
Y = tf.placeholder(dtype=tf.float32, shape=[None, nb_classes], name='Y')

In [13]:
with tf.variable_scope('softmax') as scope:
    W = tf.Variable(tf.random_normal(shape=[784, nb_classes]), shape=[784, nb_classes], dtype=tf.float32, name='weight')
    b = tf.Variable(tf.random_normal(shape=[nb_classes]), shape=[nb_classes], dtype=tf.float32, name='bias')
    logits = tf.matmul(X, W) + b
    hypothesis = tf.nn.softmax(logits)

In [14]:
with tf.variable_scope('cost') as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

In [15]:
with tf.variable_scope('Train') as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
with tf.variable_scope('accuracy') as scope:
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:
Use `tf.math.argmax` instead


In [17]:
with tf.variable_scope('tensorboard') as scope:
    overall_cost     = tf.placeholder(shape=None, dtype=tf.float32, name='overall_cost')
    overall_accuracy = tf.placeholder(shape=None, dtype=tf.float32, name='overall_accuracy')
    tf.summary.scalar('cost', overall_cost)
    tf.summary.scalar('accuracy', overall_accuracy)

In [18]:
with tf.Session(config=config) as sess:
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        _overall_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, train], feed_dict={X: batch_xs, Y: batch_ys})
            _overall_cost += cost_val / total_batch
        
        _overall_accuracy = 0
        total_batch = int(mnist.test.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.test.next_batch(batch_size)
            accuracy_val = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
            _overall_accuracy += accuracy_val / total_batch
            
        summary = sess.run(merged_summary, feed_dict={overall_cost: _overall_cost, overall_accuracy: _overall_accuracy})
        writer.add_summary(summary, global_step=(epoch))
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(_overall_cost), 'accuracy =', '{:.9f}'.format(_overall_accuracy))

Epoch: 0001 cost = 5.745170949 accuracy = 0.567199998
Epoch: 0002 cost = 1.781488078 accuracy = 0.733800001
Epoch: 0003 cost = 1.122674557 accuracy = 0.799899997
Epoch: 0004 cost = 0.871361883 accuracy = 0.832499996
Epoch: 0005 cost = 0.738527935 accuracy = 0.851699998
Epoch: 0006 cost = 0.654354536 accuracy = 0.867600002
Epoch: 0007 cost = 0.595750990 accuracy = 0.876200002
Epoch: 0008 cost = 0.552198451 accuracy = 0.884000002
Epoch: 0009 cost = 0.518347334 accuracy = 0.889299999
Epoch: 0010 cost = 0.490396257 accuracy = 0.890999998
Epoch: 0011 cost = 0.468099669 accuracy = 0.896500004
Epoch: 0012 cost = 0.448688828 accuracy = 0.898200001
Epoch: 0013 cost = 0.432459962 accuracy = 0.901400002
Epoch: 0014 cost = 0.418572140 accuracy = 0.902600000
Epoch: 0015 cost = 0.406201535 accuracy = 0.904499999
Epoch: 0016 cost = 0.395338746 accuracy = 0.904500002
Epoch: 0017 cost = 0.385045630 accuracy = 0.906700001
Epoch: 0018 cost = 0.376219346 accuracy = 0.907300003
Epoch: 0019 cost = 0.3684919